In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
import pandas as pd

In [2]:
df = pd.read_csv("authors.csv")

In [3]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [4]:
class TextDataset(Dataset): # наследует от пайторческого датасета
    def __init__(self, texts, labels, vectorizer):
        self.texts = texts
        self.labels = labels
        self.vectorizer = vectorizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts.iloc[index]
        label = self.labels.iloc[index]
        vectorized_text = self.vectorizer.transform([text]).toarray().squeeze() #вектор --> numpy --> one-dimensional
        return torch.FloatTensor(vectorized_text), torch.FloatTensor([label])

In [5]:
vectorizer = CountVectorizer()
vectorizer.fit(train_df['text'])

CountVectorizer()

In [6]:
train_dataset = TextDataset(train_df['text'], train_df['index'], vectorizer)
test_dataset = TextDataset(test_df['text'], test_df['index'], vectorizer)

In [7]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
class NeuralNet(nn.Module): # наследует от пайторческой нейронки
    def __init__(self, input_size):
        super(NeuralNet, self).__init__() # nn.овский innit
        self.layer = nn.Linear(input_size, 1) # линейная kx+b 
        self.sigmoid = nn.Sigmoid() # активация

    def forward(self, x):
        x = self.layer(x)
        x = self.sigmoid(x)
        return x

### `Neural Net --> Linear` <br>
Самая распространная и простая в имплементации моделька. Остальные, которые я посмотрела, <br>
используются для более сложных задач классификации и более объемных датасетов. <br>
Так что для этой задачи вполне подходила линейная с сигмоидой.

In [9]:
input_size = len(vectorizer.get_feature_names_out())
model = NeuralNet(input_size)
loss_count = nn.BCELoss() # Binary Cross Enthropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.01) # обновлять параметры

In [10]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad() # ставим на ноль перед каждым сэмплом
        outputs = model(inputs)
        loss = loss_count(outputs, labels)
        loss.backward() # backpropagation
        optimizer.step()
    print(f'Эпоха #{epoch+1}, Loss: {loss}')

Эпоха #1, Loss: 0.1328076869249344
Эпоха #2, Loss: 0.01439106184989214
Эпоха #3, Loss: 0.01087555568665266
Эпоха #4, Loss: 0.005815727170556784
Эпоха #5, Loss: 0.004444061778485775
Эпоха #6, Loss: 0.0049326419830322266
Эпоха #7, Loss: 0.0018565593054518104
Эпоха #8, Loss: 0.0025938290636986494
Эпоха #9, Loss: 0.0012192873982712626
Эпоха #10, Loss: 0.0014434984186664224


In [11]:
model.eval()
all_predictions = []
all_labels = []
with torch.no_grad(): # не считает градиент в оценивании точности
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predictions = (outputs >= 0.5).float() #округление
        all_predictions.extend(predictions.numpy())
        all_labels.extend(labels.numpy())

In [12]:
accuracy = accuracy_score(all_labels, all_predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9863945578231292


In [15]:
def predict_class(text):
    vectorized_text = vectorizer.transform([text]).toarray().squeeze()
    input_tensor = torch.FloatTensor(vectorized_text)
    
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
        prediction = 1 if output.item() >= 0.5 else 0

    return prediction

In [16]:
remark_text ='Женщина шла наискосок через мост прямо на Равика. Она шла  быстро,  но каким-то  нетвердым шагом. Равик заметил ее лишь  тогда, когда она оказалась почти  рядом.'
prediction = predict_class(remark_text)
print(f"Класс: {prediction}")

Класс: 0


In [18]:
my_text = 'Ввожу текст написанный мной для того чтоб оценить точность модельки'
prediction = predict_class(my_text)
print(f"Класс: {prediction}")

Класс: 0


In [19]:
krapivin_text = 'Револьвер был итальянской системы «Пикколо». В точности как настоящий. Только вместо медных патронов – стеклянные баллончики со сжатым воздухом. Хлопал он оглушительно, бил пластмассовыми пулями крепко, но застрелиться из него было все-таки невозможно. Я и не пробовал. Вместо этого лежал на тахте и стрелял по фужерам.'
prediction = predict_class(krapivin_text)
print(f"Класс: {prediction}")

Класс: 1
